- https://rapidapi.com/alexanderxbx/api/twitter-api45

In [1]:
from typing import Tuple, Literal, Optional, Any
from pathlib import Path

import pandas as pd
from pydantic import BaseModel

from twitter45.params import (
    UserTimeline, ParamsUserTimeline,
)
from scraping_kit.db.db_twitter import DBTwitter
from scraping_kit.responser import Responser
from twitter45.headers import get_headers

path_data = Path("data")
responser = Responser(headers=get_headers())
db_tw = DBTwitter(path_data, "scrape_tw")

In [2]:
df_acc = pd.read_excel("data/twitter_accounts.xlsx")
df_acc = df_acc.sort_values("followersCount", ascending=False).reset_index(drop=True)
df_acc["exists"] = ""

In [3]:
class ReqArgsHandler:
    ENDPOINT_NAME = "endpoint_name"
    _handler = {cls.endpoint_name(): cls for cls in [UserTimeline]}

    @staticmethod
    def load_req_args(req_args_json: dict):
        endpoint_name = req_args_json.pop(ReqArgsHandler.ENDPOINT_NAME)
        return ReqArgsHandler._handler[endpoint_name](**req_args_json)

In [4]:
RESPONSE_JSON = "response_json"
REQ_ARGS = "req_args"

def raw_data_to_json(raw_data: dict):
    req_args = ReqArgsHandler.load_req_args(raw_data.pop(REQ_ARGS))
    response_json = raw_data.pop(RESPONSE_JSON)
    return req_args, response_json

In [5]:
class UserSuspended(BaseModel):
    status: Literal["suspended"]
    profile: str

class User(BaseModel):
    status: Literal["active"]
    profile: str
    rest_id: str
    blue_verified: bool
    avatar: str
    header_image: str
    desc: str
    name: str
    protected: Optional[bool]
    location: str
    friends: int
    sub_count: int
    id: str


In [9]:
texts = [t["text"] for t in db_tw.coll.raw.find_one({})["response_json"]["timeline"]]


In [13]:
texts

["Imagine trying for decades to cut Social Security, Medicare, and Medicaid instead of working with us to lower the costs of insulin and prescription drugs. \n  \nThat's Congressional Republicans.",
 'After the Philly Fairmount fire, I told city officials I would do everything I could to bring Ladder 1 back into service.\n\nAs of this week, the neighborhood once again has a ladder company on call 24 hours a day, 365 days a year to keep them safe.\n\nI kept my promise. https://t.co/WTY5pLvDZR',
 "Instead of fighting to lower costs for families, Republicans in Congress are fighting to increase profits for Big Pharma.\n \nThey’re fighting to lower taxes for the wealthy and corporations.\n \nAnd they're fighting to cut Medicare and Social Security.\n \nThat’s just wrong.",
 'I’m a capitalist. \n \nI have no problem with companies making reasonable profits. \n \nBut not on the backs of seniors and working families.',
 'While Congressional Republicans fight for the wealthy and big corporatio

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd

# Preprocesamiento y Vectorización
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(texts)

# Modelado de temas con LDA
lda = LatentDirichletAllocation(n_components=5, random_state=0)
topics = lda.fit_transform(X)

# Mostrar los temas
feature_names = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    print(f"Topic #{topic_idx}:")
    print(" ".join([feature_names[i] for i in topic.argsort()[:-10 - 1:-1]]))


Topic #0:
eyes look pay people costs americans government world hardworking prices
Topic #1:
aids hearing americans making affordable took stand ll signed executive
Topic #2:
economy stands republicans corporations going keeps economic middle america fighting
Topic #3:
freedom ladder house trying american year day bring working better
Topic #4:
big pharma fighting https lower profits costs long act millions


In [ ]:
for raw_data in db_tw.coll.raw.find({}):
    req_args, response_json = raw_data_to_json(raw_data)
    user = response_json.pop("user")

    if user["status"] == "suspended":
        user = UserSuspended(status="suspended", profile=req_args.params.screenname)
    else:
        timeline = response_json.pop("timeline")
        next_cursor = response_json.pop("next_cursor")
        user = User(**user)
    display(timeline[-1])
    break

In [ ]:
db_tw.coll.user.inser

In [48]:
import requests

url = "https://twitter-api45.p.rapidapi.com/checkfollow.php"

#querystring = {"user":"elonmusk","follows":["jordanbpeterson"]}
#querystring = {"user":"DavalosAlejo","follows":"MrBeast"}
querystring = {"user":"MrBeast","follows":"DavalosAlejo"}

headers = {
	"X-RapidAPI-Key": "929a28e077msh76a649de6e684b6p1b5654jsnb2563a519484",
	"X-RapidAPI-Host": "twitter-api45.p.rapidapi.com"
}

response_2 = requests.get(url, headers=headers, params=querystring)
response_2.json()

{'is_follow': False, 'username': 'MrBeast'}

In [10]:
import requests

url = "https://twitter-api45.p.rapidapi.com/followers.php"

querystring = {"screenname": "elonmusk"}

headers = {
	"X-RapidAPI-Key": "929a28e077msh76a649de6e684b6p1b5654jsnb2563a519484",
	"X-RapidAPI-Host": "twitter-api45.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

In [49]:
response.json()

{'followers_count': 166508065,
 'followers': [{'user_id': '95092020',
   'screen_name': 'jordanbpeterson',
   'description': 'Best-Selling Author | Clinical Psychologist | #1 Education Podcast | Listen to the podcast here:',
   'profile_image': 'https://pbs.twimg.com/profile_images/1407056014776614923/TKBC60e1_normal.jpg',
   'statuses_count': 46281,
   'followers_count': 4926744,
   'friends_count': 1467,
   'media_count': 6892,
   'name': 'Dr Jordan B Peterson'},
  {'user_id': '39344374',
   'screen_name': 'DonaldJTrumpJr',
   'description': '',
   'profile_image': 'https://pbs.twimg.com/profile_images/766652495858896897/LjrJJB9a_normal.jpg',
   'statuses_count': 47660,
   'followers_count': 10765361,
   'friends_count': 1943,
   'media_count': 2664,
   'name': 'Donald Trump Jr.'},
  {'user_id': '18584875',
   'screen_name': 'hodgetwins',
   'description': 'Merch & Giveaways at: https://t.co/kxb8qjHatu Our show on Steven Crowder’s Network https://t.co/i5XwecMGFQ',
   'profile_image':

In [22]:
for f in response.json()["followers"]:
    if f["followers_count"] == 0:
        display(f)

{'user_id': '1736768014857408512',
 'screen_name': 'Moss0Narco11258',
 'description': '',
 'profile_image': 'https://pbs.twimg.com/profile_images/1736768431892951040/iRUNpmcw_normal.png',
 'statuses_count': 0,
 'followers_count': 0,
 'friends_count': 1,
 'media_count': 0,
 'name': 'narcotic moss0'}

{'user_id': '1736768246345310208',
 'screen_name': 'JollyN25134',
 'description': '',
 'profile_image': 'https://pbs.twimg.com/profile_images/1736768430538186752/rdZ80u2R_normal.jpg',
 'statuses_count': 0,
 'followers_count': 0,
 'friends_count': 3,
 'media_count': 0,
 'name': 'jolly next'}

{'user_id': '1736767753329999872',
 'screen_name': 'rodrigo28123333',
 'description': '',
 'profile_image': 'https://pbs.twimg.com/profile_images/1736768235863781377/Ix0uJVkH_normal.jpg',
 'statuses_count': 0,
 'followers_count': 0,
 'friends_count': 1,
 'media_count': 0,
 'name': 'rodrigo'}

{'user_id': '1736767120195698688',
 'screen_name': 'StephanieC87636',
 'description': '',
 'profile_image': 'https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png',
 'statuses_count': 0,
 'followers_count': 0,
 'friends_count': 1,
 'media_count': 0,
 'name': 'Stephanie Cook'}

{'user_id': '1736768237612756992',
 'screen_name': 'bonnie91204',
 'description': '',
 'profile_image': 'https://pbs.twimg.com/profile_images/1736768429254737920/7tARq9yp_normal.jpg',
 'statuses_count': 0,
 'followers_count': 0,
 'friends_count': 3,
 'media_count': 0,
 'name': 'Bonnie Smith'}

...

{'user_id': '1736768112404430848',
 'screen_name': 'Ye4presid3nt',
 'description': '',
 'profile_image': 'https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png',
 'statuses_count': 0,
 'followers_count': 0,
 'friends_count': 2,
 'media_count': 0,
 'name': 'Lemo'}

{'user_id': '1736767957152288768',
 'screen_name': 'HakimMahzoul',
 'description': '',
 'profile_image': 'https://pbs.twimg.com/profile_images/1736768159544119297/ks6UVw46_normal.png',
 'statuses_count': 0,
 'followers_count': 0,
 'friends_count': 1,
 'media_count': 0,
 'name': 'Hakim MAHZOUL'}

{'user_id': '1736767806748651520',
 'screen_name': 'BluntHeidi42139',
 'description': '',
 'profile_image': 'https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png',
 'statuses_count': 0,
 'followers_count': 0,
 'friends_count': 3,
 'media_count': 0,
 'name': 'Heidi Blunt'}

{'user_id': '1736768154758770688',
 'screen_name': 'idohyeon270135',
 'description': '',
 'profile_image': 'https://pbs.twimg.com/profile_images/1736768384828874752/oRpP5lPp_normal.png',
 'statuses_count': 0,
 'followers_count': 0,
 'friends_count': 0,
 'media_count': 0,
 'name': '이도현'}

{'user_id': '1699458988323037188',
 'screen_name': 'ELISA5867597058',
 'description': '',
 'profile_image': 'https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png',
 'statuses_count': 1,
 'followers_count': 0,
 'friends_count': 5,
 'media_count': 0,
 'name': 'ELISA'}

# User replies

In [73]:
import requests

url = "https://twitter-api45.p.rapidapi.com/replies.php"

querystring = {"screenname":"elonmusk"}

headers = {
	"X-RapidAPI-Key": "929a28e077msh76a649de6e684b6p1b5654jsnb2563a519484",
	"X-RapidAPI-Host": "twitter-api45.p.rapidapi.com"
}

response_3 = requests.get(url, headers=headers, params=querystring)

In [80]:
response_3.json()["timeline"][-1]

{'tweet_id': '1736531085939404880',
 'bookmarks': 5,
 'created_at': 'Sun Dec 17 23:37:28 +0000 2023',
 'favorites': 483,
 'text': '@GavinSBaker Yeah',
 'lang': 'en',
 'views': '37559',
 'quotes': 3,
 'replies': 43,
 'retweets': 33,
 'conversation_id': '1736526232022376628',
 'media': [],
 'author': {'rest_id': '44196397',
  'name': 'Elon Musk',
  'screen_name': 'elonmusk',
  'avatar': 'https://pbs.twimg.com/profile_images/1683325380441128960/yRsRRjGO_normal.jpg',
  'blue_verified': True}}

# following

In [81]:
import requests

url = "https://twitter-api45.p.rapidapi.com/following.php"

querystring = {"screenname":"elonmusk"}

headers = {
	"X-RapidAPI-Key": "929a28e077msh76a649de6e684b6p1b5654jsnb2563a519484",
	"X-RapidAPI-Host": "twitter-api45.p.rapidapi.com"
}

response_4 = requests.get(url, headers=headers, params=querystring)
response_4.json().keys()

dict_keys(['following', 'next_cursor', 'status', 'more_users'])

In [85]:
response_4.json()["following"].__len__()

69

# favorites

In [86]:
import requests

url = "https://twitter-api45.p.rapidapi.com/favorites.php"

querystring = {"id": "1703348962697396461"}

headers = {
	"X-RapidAPI-Key": "929a28e077msh76a649de6e684b6p1b5654jsnb2563a519484",
	"X-RapidAPI-Host": "twitter-api45.p.rapidapi.com"
}

response_5 = requests.get(url, headers=headers, params=querystring)
response_5.json().keys()

dict_keys(['favorites', 'cursor'])

In [92]:
len(response_5.json()["favorites"])

20

In [ ]:
broken_responses = []
for i, row in df_acc.iterrows():
    if i<=49:
        continue
    screenname = row["screenName"]
    ######## --> profile_url = row["profileUrl"]

    params = ParamsUserTimeline(screenname=screenname, cursor="")
    req_args = UserTimeline(params=params)
    response = responser.get_response(req_args)
    try:
        response_json = response.json()
        db_tw.coll.raw.insert_one({
            "endpoint": "user_timeline",
            "params": req_args.model_dump(),
            "response_json": response_json
        })
        df_acc.loc[i, "exists"] = "si"
    except:
        print(screenname)
        df_acc.loc[i, "exists"] = "no"
        broken_responses.append({
            "i": i,
            "screenname": screenname,
            "response": response,
            "req_args": req_args
        })
    if i==49:
        break

- 50 -> 3m 9.4seg
- 3.78s/requests

In [ ]:
len(broken_responses)

In [ ]:
import requests
import time

base_url = "https://twitter-api45.p.rapidapi.com/following.php"
headers = {
    'X-RapidAPI-Key': "929a28e077msh76a649de6e684b6p1b5654jsnb2563a519484",
    'X-RapidAPI-Host': "twitter-api45.p.rapidapi.com"
}

def get_response(base_url, headers, params):
    return requests.get(base_url, headers=headers, params=params)

def get_followers(screen_name, rapidapi_key):
    cursor = ""
    following_array = []

    while True:
        params = {'screenname': screen_name, 'cursor': cursor}
        response = get_response(base_url, headers, params)
        followers_data = response.json()
        
        following_array.append(followers_data['following'])
        cursor = followers_data.get('next_cursor')
        
        if not cursor or not followers_data['following']:
            break
        
        time.sleep(1)  # Respect API rate limits

    return following_array

rapidapi_key = "929a28e077msh76a649de6e684b6p1b5654jsnb2563a519484"
screen_name = "zachxbt"
followers_list = get_followers(screen_name, rapidapi_key)


# _

Hi Vera,

I have some exciting news to share with you. Please allow me one additional day to finalize the integration, as I have discovered an API that is both cost-effective and feature-rich. This API offers an incredible deal at only $5 per month for 100,000 requests with no limit on the number of requests per second, providing the same information we need.

I just need to adapt the data extraction to this new source, which won’t require significant changes due to the scalability of my existing code. This is perfect, especially considering the volume of accounts we are dealing with:

- We can retrieve the profile and latest tweets for each account, which instantly gives us valuable insights into account activity and interests.
- For those accounts with a substantial follower base, we can fetch all followers using the API's pagination feature. While this is computationally demanding, the affordability of the API at $5 allows us to consider parallelizing requests across multiple accounts.

I apologize for the shifts in direction; such adjustments are part of the process as we delve deeper into the project and test our approaches.

I'm thrilled about this discovery and plan to make the most of the 1,000 free requests available to us for testing purposes today.

Thank you for your patience and understanding as we navigate through these optimizations.


In [ ]:
"""
from pymongo import MongoClient

for doc in db_tw.coll.raw.find():
    endpoint_value = doc.get('endpoint')
    params_value = doc.get('params')

    params_value['endpoint_name'] = endpoint_value

    # Preparar la actualización
    new_values = {
        "$set": {
            "req_args": params_value  # Cambiar 'params' a 'req_args'
        },
        "$unset": {
            "endpoint": "",  # Eliminar el campo 'endpoint'
            "params": ""  # Eliminar el campo 'params' original
        }
    }

    # Actualizar el documento
    db_tw.coll.raw.update_one({"_id": doc['_id']}, new_values)
"""

In [ ]:
'''
from concurrent.futures import ThreadPoolExecutor, as_completed
import random
import time

def some_function(i: int) -> dict:
    time.sleep(2)
    return {"i": i}

max_workers = 100

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = (executor.submit(some_function, i) for i in range(100))

    for future in as_completed(futures):
        result = future.result()
        print(result)
'''